In [ ]:
import time
# my_list = [time.sleep(x) for x in range(1,3)]
my_list = (time.sleep(x) for x in range(1,3))
print(my_list)

In [ ]:
import sys
my_list = (x for x in range(1000000000))

In [ ]:
print(sys.getsizeof(my_list))

In [ ]:
my_range = range(1,10)
print(list(my_range))

In [ ]:
def my_lazy_func():
    for i in range(1,11):
        print(f"before{i}")
        yield i
        print(f"after{i}")


for i in my_lazy_func():
    print(i)

In [ ]:
# def my_lazy_func():
#     for i in items:
#         yield i

def my_lazy_func():
    yield from items
        

items = ["apple","banana","tomato","orange"]
for i in my_lazy_func():
    print(i)

In [ ]:
# my_file = open("requirements.txt","r")
# a = my_file.read()
# my_file.close()
# print(a)
with open("requirements.txt","r") as my_file:
    a = my_file.read()
print(a)
print(my_file.closed)

In [31]:
import contextlib

@contextlib.contextmanager
def str_reverse(my_str):
    print("Entering context manager")
    yield my_str[::-1]
    print("Exit context manager")
with str_reverse("hello world") as reversed_str:
    print(f"Our string {reversed_str}")

Entering context manager
Our string dlrow olleh
Exit context manager


In [32]:
@contextlib.contextmanager
def exc_handler(exc):
    try:
        yield True
    except exc:
        print("EXEPTION")
        
with exc_handler(IndexError):
    my_list = [1,2,3]
    print(my_list[3])

EXEPTION


In [38]:
def summator(*args):
    print(args)
    answear = 0
    for i in args:
        answear += i
    return answear
a = []
for i in range(1,4):
    s = input()
    if s.isdigit():a.append(int(s))
print(a)
print(summator(*a))

[1, 3]
(1, 3)
4


In [39]:
def summator(**kwargs):
    answear = 0
    for i in kwargs:
        answear += kwargs[i]
    return answear
print(summator(a=5,b=6))

11
